In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

In [3]:
# read data
with open(r"C:\Users\Gokul\Downloads\reviews.txt", 'r') as f:
    reviews = f.read()
with open(r"C:\Users\Gokul\Downloads\labels.txt", 'r') as f:
    labels = f.read()

print(reviews[:1000])
print(labels[:20])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

In [8]:
from string import punctuation

# changing all text to lowercase and removing punctuation
reviews = reviews.lower()
all_text = ''.join([c for c in reviews if c not in punctuation ])
# splitting using \n
review_split = all_text.split('\n')
all_text = ''.join(review_split)
#split using white spaces
words = all_text.split()
print(words[:20])

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', 'such']


In [10]:
from collections import Counter

counts = Counter(words)
#print(counts)
vocab = sorted(counts, key=counts.get, reverse=True)
#print(vocab)
vocab2int = {word: i for i, word in enumerate(vocab, 1) } #1 given to start index from 1
#print(vocab2int)

# converting words in review to int
reviews_ints = []
for review in review_split:
    reviews_ints.append([vocab2int[word] for word in review.split()])

In [5]:
# stats about vocabulary
print('Unique words: ', len((vocab2int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', reviews_ints[:1])

Unique words:  74072

Tokenized review: 
 [[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]]


In [6]:
# 1=positive, 0=negative label conversion
labels = labels.lower()
labels_split = labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])
print(labels_split)
print(encoded_labels)

['positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'ne

In [7]:
# Outliers review
review_lengths = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lengths[0]))
print("Maximum review length: {}".format(max(review_lengths)))

Zero-length reviews: 1
Maximum review length: 2514


In [8]:
print('Number of reviews before removing outliers: ', len(reviews_ints))

#remove any reviews/labels with zero length from the reviews_ints list.
non_zero_idx = [i for i, review in enumerate(reviews_ints) if len(review) != 0]
#print(non_zero_idx)
reviews_ints = [reviews_ints[i] for i in non_zero_idx]
encoded_labels = np.array([encoded_labels[i] for i in non_zero_idx])

print('Number of reviews after removing outliers: ', len(reviews_ints))

Number of reviews before removing outliers:  25001
Number of reviews after removing outliers:  25000


In [9]:
def pad_features(reviews_ints, seq_length):
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]

    return features

In [10]:
# Testing pad_features function

seq_length = 200
features = pad_features(reviews_ints, seq_length=seq_length)

## test statements
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches
print(features[:30,:10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34     3   162   748  2731     9   325]
 [    9    11 10171  5305  1946   689   444    22   280   673]
 [    0     0     0     0     0     0     0     0     0

In [11]:
split_frac = 0.8
split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
valid_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
valid_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

print("Train set: \t\t{}".format(train_x.shape),
      "\nValidation set: \t{}".format(valid_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


In [12]:
# Creating Tensor Datasets
train_data = TensorDataset(torch.from_numpy(train_x),torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(valid_x),torch.from_numpy(valid_y))
test_data = TensorDataset(torch.from_numpy(test_x),torch.from_numpy(test_y))

# Dataloaders
batch_size = 50

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [13]:
# obtain one batch of training data
example = iter(train_loader)
sample_x, sample_y = example.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[ 387,   10,  143,  ...,   11,  921,   19],
        [   0,    0,    0,  ...,   24,    5,   39],
        [  11,   20,   14,  ...,   45,   16,   48],
        ...,
        [  10, 1668, 3021,  ...,    9,   11,   20],
        [   0,    0,    0,  ..., 1589,   17,    8],
        [ 101,  176,  320,  ..., 3232, 1522, 2225]], dtype=torch.int32)

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        1, 0], dtype=torch.int32)


In [14]:
# Sentiment RNN
class RNN(nn.Module):
    def __init__(self, vocab_size, output_size, embed_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(RNN, self).__init__()
        self.output_size = output_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Embedding and LSTM
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden):
        batch_size = x.size(0)

        # Embedding and LSTM output
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        # Dropout and FC after reshaping
        out = self.dropout(lstm_out)
        out = self.fc(out)
        sig_out = self.sig(out)
        # Reshape to batch size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels

        return sig_out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

In [15]:
# Hyperparameters
vocab_size = len(vocab2int)+1 # vocab size + 1 for 0 padding
output_size = 1
embed_dim = 400
hidden_dim = 256
n_layers = 2
epochs = 1
lr = 0.001

model = RNN(vocab_size, output_size, embed_dim, hidden_dim, n_layers)
print(model)

RNN(
  (embedding): Embedding(74073, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [16]:
# Loss and optimization
loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [17]:
counter = 0
print_every = 25
clip = 5

# Training loop
model.train()
for epoch in tqdm(range(epochs)):
    h = model.init_hidden(batch_size)

    for inputs, labels in train_loader:
        counter += 1
        h = tuple([each.data for each in h])

        output, h = model(inputs, h)
        loss = loss_func(output.squeeze(), labels.float())

        model.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        # Loss stats
        if counter % print_every == 0:
            # Validation loss
            valid_h = model.init_hidden(batch_size)
            valid_losses = []
            model.eval()
            for inputs, labels in valid_loader:
                valid_h = tuple([each.data for each in valid_h])
                output, valid_h = model(inputs, valid_h)
                valid_loss = loss_func(output.squeeze(), labels.float())
                valid_losses.append(valid_loss.item())

            model.train()
            print('Epoch: {}/{}...'.format(epoch+1, epochs),
                  'Step: {}...'.format(counter),
                  'Loss: {:.6f}...'.format(loss.item()),
                  'Valid_loss: {:.6f}...'.format(np.mean(valid_losses)))

100%|██████████| 1/1 [24:39<00:00, 1479.20s/it]


Epoch: 1/1... Step: 25... Loss: 0.686892... Valid_loss: 0.686705...
Epoch: 1/1... Step: 50... Loss: 0.692456... Valid_loss: 0.661529...
Epoch: 1/1... Step: 75... Loss: 0.610167... Valid_loss: 0.663541...
Epoch: 1/1... Step: 100... Loss: 0.622511... Valid_loss: 0.664822...
Epoch: 1/1... Step: 125... Loss: 0.695463... Valid_loss: 0.668663...
Epoch: 1/1... Step: 150... Loss: 0.639237... Valid_loss: 0.625974...
Epoch: 1/1... Step: 175... Loss: 0.581800... Valid_loss: 0.627034...
Epoch: 1/1... Step: 200... Loss: 0.585396... Valid_loss: 0.608546...
Epoch: 1/1... Step: 225... Loss: 0.668543... Valid_loss: 0.642465...
Epoch: 1/1... Step: 250... Loss: 0.613473... Valid_loss: 0.603990...
Epoch: 1/1... Step: 275... Loss: 0.527240... Valid_loss: 0.568935...
Epoch: 1/1... Step: 300... Loss: 0.527939... Valid_loss: 0.599428...
Epoch: 1/1... Step: 325... Loss: 0.561088... Valid_loss: 0.696735...
Epoch: 1/1... Step: 350... Loss: 0.616147... Valid_loss: 0.619550...
Epoch: 1/1... Step: 375... Loss: 0.44

In [21]:
# Testing

test_losses = []
num_correct = 0

h = model.init_hidden(batch_size)
model.eval()

for inputs, labels in test_loader:
    h = tuple([each.data for each in h])
    output, h = model(inputs, h)
    test_loss = loss_func(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())

    # converting output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer

    # compare pred with true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy())
    num_correct += np.sum(correct)

# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.627
Test accuracy: 0.616


In [25]:
# Inference

def tokenize_review(test_review):
    test_review = test_review.lower()
    test_text = ''.join([c for c in test_review if c not in punctuation])
    test_words = test_text.split()

    test_ints = []
    test_ints.append([vocab2int[word] for word in test_words])

    return test_ints

In [26]:
def predict(model, test_review, sequence_length = 200):
    model.eval()
    test_ints = tokenize_review(test_review)
    seq_length = sequence_length
    features = pad_features(test_ints, seq_length)
    feature_tensor = torch.from_numpy(features)

    batch_size = feature_tensor.size(0)
    h = model.init_hidden(batch_size)
    output, h = model(feature_tensor, h)
    pred = torch.round(output.squeeze())

    # printing output value, before rounding
    print('Prediction value, pre-rounding: {:.6f}'.format(output.item()))
    # print response
    if(pred.item()==1):
        print("Positive review detected!")
    else:
        print("Negative review detected.")

In [27]:
# negative test review
test_review_neg = 'The worst movie I have seen; acting was terrible and I want my money back. This movie had bad acting and the dialogue was slow.'

# call function
seq_length = 200 # good to use the length that was trained on
predict(model, test_review_neg, seq_length)

Prediction value, pre-rounding: 0.286948
Negative review detected.


In [34]:
# positive test review
test_review_pos = 'This movie fulfills all the criteria required for a blockbuster in terms of story, message, performances and execution'
predict(model, test_review_pos, seq_length)

Prediction value, pre-rounding: 0.892452
Positive review detected!
